# Project: Data merging

Recall our goal is to produce a plot that explores the relationship between carbon dioxide emissions, income, and population for each country/territory. In this step of the project, we'll merge our four data sources together into a single DataFrame that includes:

1. Carbon dioxide emissions by country
2. Income (as measured by GDP per capita) by country
3. Population by country (so we can convert CO2 emissions into per capita emissions)
4. List of territories by continent (since we want to be able to group the countries by region of the world)

We've already saved the individual datasets we need: `co2`, `income`, `pop` and `continent` and created a series of steps that we could reuse if we need to process a different year of data. Now, we need to merge these into one `DataFrame` so that we can plot the data. 

## The goal

The goal for this part of the project is simple: merge the data into a single DataFrame.


## The process

**(1)** Review each of the datasets you've processed and think about what variable you will merge on (it should be present in all four separate datasets). From this, it should be clear that you want to merge on country, since this is common across all the datasets. You'll want to have one row for each country/region with the values of carbon dioxide, income, population, and continent as columns. Collect any insights about the similarities and differences between the country values across the datasets.

**(2)** Merge data with similar country values. In this case, since the same organization processed the carbon dioxide, income, and population data, you should be able to merge each of these without issue. Start with those three datasets and merge them. Determine what type of merge is most appropriate given the data.

**(3)** Compare the country names between the new dataset from the three merged datasets in the last step and the continents dataset.

**(4)** In the last few steps, you likely noticed that country names are not identical across the datasets if you look closely. Examples of the differences include:

- 'Brunei', 'Brunei Darussalam'
- 'Bolivia', 'Bolivia (Plurinational State of)'
- 'Venezuela','Venezuela (Bolivarian Republic of)'
- 'St. Vincent and the Grenadines', 'Saint Vincent and the Grenadines'
- 'Cote d'Ivoire', 'Côte d’Ivoire'

You will need to reconcile these differences. Create a method for identifying matching pairs of all country names that we can confidently match, and manually label the remaining territories. You will have to consider how to handle 4 categories of possible matches (or lack thereof):
1. **Exact matches**. Character-for-character equality. E.g., 'Burundi' <-> 'Burundi'
2. **Subset matches**. The "shorthand" name is a subset of the full name. E.g., 'Bolivia' <-> 'Bolivia (Plurinational State of)'
3. **Near matches**. The names are VERY similar, but a few characters may be different. E.g., 'Cape Verde' <-> 'Cabo Verde'
4. **Unmatched examples**. Cases that do not have corresponding pairs. E.g. 'Holy See'

Develop a solution for handling each of the above cases and apply them to your data. Once enough of the countries are matched, merge the data. You may need to code a manual solution for a handful of locations to set the content. With programming in data science, a human is often in the loop, but we try to always make sure our solution can be automatically reproduced as needed.

Think through how you would solve this problem before proceeding.

**(5)** As a recommended approach, use the list of 4 categories of match types above and test for each. What is important here is that order matters. If you perform a match, the matched strings should be removed from further consideration going forward, since they've been matched, and so you don't create duplicate matches. Additionally, match those quantities that most likely match first (e.g. exact matches before subset matches, before near matches, etc.). Since the lengths of the territory lists are 193 and 249, at most there will be 193 matches.

You're encouraged to read about and use the `difflib` package and the `SequenceMatcher` object and `get_close_matches` function. `SequenceMatcher` provides a similarity score for comparing two strings and takes on values between 0 and 1 where 1 is an exact match and 0 is no match at all. The `get_close_matches` function provides a list of the closest matches to the input string based on the `SequenceMatcher` similarity score.

Our suggested pseudocode for this function is:
1. Identify any exact matches and remove them from the list of strings to match
2. Identify subset matches and remove them from the list of strings to match
3. One at a time, create near matches based on the similarity score of each possible match
4. Repeat (3) until all 193 of the territories have been matched
5. Return the list of tuples with each match and its corresponding similarity score (set all exact matches and subset matches to a similarity score of 1)
6. Determine a threshold for the similarity score, below which assigning a match is not correct
7. Manually assign the content for the unmatched examples

Review the output of your matching process for correctness before proceeding.

**(6)** If you notice in the original plot that we're working to reproduce, there were four continents plotted ('Asia', 'Africa', 'Europe', 'Americas'), but our data has 5 ('Asia', 'Africa', 'Europe', 'Americas', 'Oceania'). Replace the Oceania label with Asia for the sake of this project.

**(5)** Save your data to file.

Coding for data science often involves a lot of preparatory work to ready the data for analysis and visualization. You've taken a big step here and now we can reap the benefits of this hard work through the visualization in the next step of the project!

**BEFORE PROCEEDING IN THE COURSE, BE SURE TO WORK THROUGH THIS YOURSELF. YOU CAN DO THIS!**

In [24]:
import pandas as pd

# Load the data
co2 = pd.read_csv('data/intermediate/co2.csv')
income = pd.read_csv('data/intermediate/income.csv')
pop = pd.read_csv('data/intermediate/pop.csv')
continent = pd.read_csv('data/intermediate/continent.csv')

In [25]:
co2

,country,co2
0,Afghanistan,0.254
1,Angola,1.120
2,Albania,1.590
3,Andorra,6.120
4,United Arab Emirates,21.400
...,...,...
189,Samoa,1.320
190,Yemen,0.356
191,South Africa,8.100
192,Zambia,0.302


In [26]:
income

,country,income
0,Afghanistan,2030.0
1,Angola,6930.0
2,Albania,13300.0
3,Andorra,58300.0
4,United Arab Emirates,67000.0
...,...,...
190,Samoa,6390.0
191,Yemen,2660.0
192,South Africa,13900.0
193,Zambia,3520.0


In [27]:
pop

,country,population
0,Afghanistan,37200000.0
1,Angola,30800000.0
2,Albania,2880000.0
3,Andorra,77000.0
4,United Arab Emirates,9630000.0
...,...,...
192,Samoa,196000.0
193,Yemen,28500000.0
194,South Africa,57800000.0
195,Zambia,17400000.0


In [28]:
continent

,continent,country
0,Asia,Afghanistan
1,Europe,Albania
2,Africa,Algeria
3,Oceania,American Samoa
4,Europe,Andorra
...,...,...
244,Africa,Western Sahara
245,Asia,Yemen
246,Africa,Zambia
247,Africa,Zimbabwe


The first observation when looking across these four datasets is that they each have a different number of rows! 194, 195, 197, and 249, respectively. This means they don't all contain the same countries. But we need to combine them to plot them.

Think about what type of merge would be appropriate here? Left, right, inner, or outer? We only want to save the rows that are matching, so an inner merge seems like the right approach. Let's start by merging co2 and income based on the 'country' column:

In [29]:
data = pd.merge(co2, income, how="inner", on="country")
data

,country,co2,income
0,Afghanistan,0.254,2030.0
1,Angola,1.120,6930.0
2,Albania,1.590,13300.0
3,Andorra,6.120,58300.0
4,United Arab Emirates,21.400,67000.0
...,...,...,...
188,Samoa,1.320,6390.0
189,Yemen,0.356,2660.0
190,South Africa,8.100,13900.0
191,Zambia,0.302,3520.0


Excellent - we have 193 rows, which is as expected given the smallest size among the two dataframes being merged. Next lets add in population:

In [30]:
data = pd.merge(data, pop, how="inner", on="country")
data

,country,co2,income,population
0,Afghanistan,0.254,2030.0,37200000.0
1,Angola,1.120,6930.0,30800000.0
2,Albania,1.590,13300.0,2880000.0
3,Andorra,6.120,58300.0,77000.0
4,United Arab Emirates,21.400,67000.0,9630000.0
...,...,...,...,...
188,Samoa,1.320,6390.0,196000.0
189,Yemen,0.356,2660.0,28500000.0
190,South Africa,8.100,13900.0,57800000.0
191,Zambia,0.302,3520.0,17400000.0


And again we have 193 merged columns - looks like we have everything except for the continent labels added in. Adding in the continent labels is a little bit trickier, however, because if we look at the lists of the country/territory names in our `data` and `continent` frames, we can see that some of the names that are the same are simply written a bit differently. For example the following pairs represent the same territory:

- 'Brunei', 'Brunei Darussalam'
- 'Bolivia', 'Bolivia (Plurinational State of)'
- 'Venezuela','Venezuela (Bolivarian Republic of)'
- 'St. Vincent and the Grenadines', 'Saint Vincent and the Grenadines'
- 'Cote d'Ivoire', 'Côte d’Ivoire'

This means that these will not be merged together using the `merge()` method because their values are not identical.

We will create a method for idnetifying matching pairs of all names that we can confidently match, and will manually label the continent for the remaining territories. Looking at the data, it looks like we have 4 general categories:
1. **Exact matches**. Character-for-character equality. E.g., 'Burundi' <-> 'Burundi'
2. **Subset matches**. The "shorthand" name is a subset of the full name. E.g., 'Bolivia' <-> 'Bolivia (Plurinational State of)'
3. **Near matches**. The names are VERY similar, but a few characters may be different. E.g., 'Cape Verde' <-> 'Cabo Verde'
4. **Unmatched examples**. Cases that do not have corresponding pairs. E.g. 'Holy See'

Since the lists to be merged are of length 193 and 249, respectively, then a simple approach to merging them is to manually correct the names of countries/territories in the `continents` frame to match the rest. This approach is fine, however, this is not practical for larger lists which will commonly be encountered. Instead, we will find a way to minimize the need for manual intervention and identify the nearest match from one list to the other by considering the four possible cases above. As with many aspects of data science, we cannot always eliminate the need for some manual effort in preparing the data, but we will reduce it greatly.

**EXERCISE: Create a function that takes as input two arrays of strings and returns a list the most likely matches**


In [31]:
def get_matches(list1, list2):
    pass
    # Returns a list of tuples of matches:
    #   [ (match_string_1, match_string_2, similarity_score)]

As a recommended approach, use the list of 4 categories above and test for each. What is important here is that order matters. If you perform a match, the matched strings should be removed from further consideration going forward so you don't create duplicate matches. Additionally, match those quantities that the most likely matches first (e.g. exact matches before subset matches, before near matches). Since the lengths of the territory lists are 193 and 249, at most there will be 193 matches.

You're encouraged to read about and use the `difflib` package and the `SequenceMatcher` object and `get_close_matches` function. `SequenceMatcher` provides a similarity score for comparing two strings and takes on values between 0 and 1 where 1 is an exact match and 0 is no match at all. The `get_close_matches` function provides a list of the closest matches to the input string based on the `SequenceMatcher` similarity score.

Therefore, pseudocode for this function should be:
1. Identify any exact matches and remove them from the list of strings to match
2. Identify subset matches and remove them from the list of strings to match
3. One at a time, create near matches based on the similarity score of each possible match
4. Repeat (3) until all 193 of the territories have been matched
5. Return the list of tuples with each match and its corresponding similarity score (set all exact matches and subset matches to a similarity score of 1)
6. Determine a threshold for the similarity score, below which assigning a match is not correct

In [32]:
# BREAK HERE AND DIRECT STUDENTS TO GRADED LAB

In [33]:
from difflib import SequenceMatcher, get_close_matches


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def closest(a, B):
    return get_close_matches(a, list(B), n=1, cutoff=0)


long = list(continent["country"].values)
short = list(data["country"].values)
long_strings = long.copy()
short_strings = short.copy()

# First pass, match those with exact subsets
matches = []
for ss in short:
    candidates = []
    for ls in long:
        if ss in ls:
            candidates.append(ls)
    if len(candidates) > 0:
        selected = closest(ss, candidates)[0]
        sim = similar(ss, selected)
        long_strings.remove(selected)
        short_strings.remove(ss)
        matches.append([ss, selected, 1])

# Second pass, match what remains using proximity
slength = len(short_strings)
while slength > 0:
    matching_options = []
    similarity = []
    for ss in short_strings:
        selected = closest(ss, long_strings)[0]
        similarity.append(similar(ss, selected))
        matching_options.append([ss, selected])
    x = 1
    max_index = similarity.index(max(similarity))
    sim = max(similarity)
    long_strings.remove(matching_options[max_index][1])
    short_strings.remove(matching_options[max_index][0])
    matches.append(
        [matching_options[max_index][0], matching_options[max_index][1], sim]
    )
    slength = len(short_strings)

for e in matches:
    print(f"SS = {e[0]}, LS = {e[1]}, Sim = {e[2]}")

SS = Afghanistan, LS = Afghanistan, Sim = 1
SS = Angola, LS = Angola, Sim = 1
SS = Albania, LS = Albania, Sim = 1
SS = Andorra, LS = Andorra, Sim = 1
SS = United Arab Emirates, LS = United Arab Emirates, Sim = 1
SS = Argentina, LS = Argentina, Sim = 1
SS = Armenia, LS = Armenia, Sim = 1
SS = Antigua and Barbuda, LS = Antigua and Barbuda, Sim = 1
SS = Australia, LS = Australia, Sim = 1
SS = Austria, LS = Austria, Sim = 1
SS = Azerbaijan, LS = Azerbaijan, Sim = 1
SS = Burundi, LS = Burundi, Sim = 1
SS = Belgium, LS = Belgium, Sim = 1
SS = Benin, LS = Benin, Sim = 1
SS = Burkina Faso, LS = Burkina Faso, Sim = 1
SS = Bangladesh, LS = Bangladesh, Sim = 1
SS = Bulgaria, LS = Bulgaria, Sim = 1
SS = Bahrain, LS = Bahrain, Sim = 1
SS = Bahamas, LS = Bahamas, Sim = 1
SS = Bosnia and Herzegovina, LS = Bosnia and Herzegovina, Sim = 1
SS = Belarus, LS = Belarus, Sim = 1
SS = Belize, LS = Belize, Sim = 1
SS = Bolivia, LS = Bolivia (Plurinational State of), Sim = 1
SS = Brazil, LS = Brazil, Sim = 1
S

In [34]:
for match in matches:
    if match[2] > 0.5:
        continent = continent.replace(match[1], match[0])

In [35]:
data = pd.merge(data, continent, how="left", on="country")
data

,country,co2,income,population,continent
0,Afghanistan,0.254,2030.0,37200000.0,Asia
1,Angola,1.120,6930.0,30800000.0,Africa
2,Albania,1.590,13300.0,2880000.0,Europe
3,Andorra,6.120,58300.0,77000.0,Europe
4,United Arab Emirates,21.400,67000.0,9630000.0,Asia
...,...,...,...,...,...
188,Samoa,1.320,6390.0,196000.0,Oceania
189,Yemen,0.356,2660.0,28500000.0,Asia
190,South Africa,8.100,13900.0,57800000.0,Africa
191,Zambia,0.302,3520.0,17400000.0,Africa


In [36]:
data[data.isnull().any(axis=1)]

,country,co2,income,population,continent
35,"Congo, Dem. Rep.",0.0243,1090.0,84100000.0,NaN
72,"Hong Kong, China",5.8600,61100.0,7370000.0,NaN
90,Kyrgyz Republic,1.6000,5130.0,6300000.0,NaN
94,South Korea,12.9000,42000.0,51200000.0,NaN
140,North Korea,1.1800,1860.0,25500000.0,NaN
177,Taiwan,11.6000,51900.0,23700000.0,NaN


In [37]:
continent["continent"].unique()

array(['Asia', 'Europe', 'Africa', 'Oceania', 'Americas', nan],
      dtype=object)

In [38]:
data.at[35, "continent"] = "Africa"
data.at[72, "continent"] = "Asia"
data.at[90, "continent"] = "Asia"
data.at[94, "continent"] = "Asia"
data.at[140, "continent"] = "Asia"
data.at[177, "continent"] = "Asia"
data

,country,co2,income,population,continent
0,Afghanistan,0.254,2030.0,37200000.0,Asia
1,Angola,1.120,6930.0,30800000.0,Africa
2,Albania,1.590,13300.0,2880000.0,Europe
3,Andorra,6.120,58300.0,77000.0,Europe
4,United Arab Emirates,21.400,67000.0,9630000.0,Asia
...,...,...,...,...,...
188,Samoa,1.320,6390.0,196000.0,Oceania
189,Yemen,0.356,2660.0,28500000.0,Asia
190,South Africa,8.100,13900.0,57800000.0,Africa
191,Zambia,0.302,3520.0,17400000.0,Africa


In [39]:
data.isnull().any()

country       False
co2           False
income        False
population    False
continent     False
dtype: bool

In [40]:
data[data.isna().any(axis=1)]

,country,co2,income,population,continent


In [41]:
data[data.isnull().any(axis=1)]

,country,co2,income,population,continent


Wonderful! We have our dataset successfully merged. 


## Changing the data from being divided into continents to 4 global regions

We just have one final data processing step before we create our plot. If you look at the chart on GapMinder's website, instead of using 7 continent categories, they use four categories for the world grouping Asia and Oceania into Asia. Let's do the same here by replacing any instances of "Oceania" with "Asia". Here we can use the `replace()` method.

In [42]:
data["continent"].unique()

array(['Asia', 'Africa', 'Europe', 'Americas', 'Oceania'], dtype=object)

In [43]:
data["continent"] = data["continent"].replace("Oceania", "Asia")
data["continent"].unique()

array(['Asia', 'Africa', 'Europe', 'Americas'], dtype=object)

Phew! We have done a LOT of data processing to get ready to make our plot, which we'll do in the next lesson. Our data are the fuel for all the analyses we do in data science and often its takes some effort to prepare them for use. Having a clean dataset like we do now is entirely worth it as we can now explore these data by plotting them!

In [44]:
data.to_csv('data/intermediate/data.csv', index=False)

In [45]:
### Not for inclusion - just to save contents from this notebook
# import pickle

# pickle_path = "data/pickle/"

# data.to_pickle(f"{pickle_path}data.pkl")